# Iterative Methods


For large systems of linear equations, and in particular for sparse systems (few non-zero elements), and if the matrix is _strictly diagonally dominant_ , the solution can be computed using __iterative methods__
(see [Numerička matematika, section 3.8](http://www.mathos.unios.hr/pim/Materijali/Num.pdf)):

__Definition.__ Function $F:\mathbb{R}^n\to \mathbb{R}^n$ is a __contraction__ if there is $q<1$ such that

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.$$

__Banach Fixed Point Theorem.__
If $F$ is a contraction, then the sequence defined by

$$ 
x_{k+1}=F(x_k)$$

converges towards the unique vector $\tilde x$ for which

$$
\tilde x = F(\tilde x).$$

$\tilde x$ is the  __fixed point__ of the function $F$. Error in the  $k$-th step satisfies the bounds

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|$$

and

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,$$

where the second bound is better. The speed of convergence is __linear__ ,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.$$


## Jacobi and  Gauss-Seidel methods

Let

$$F(x)=Bx+c,$$

where $B$ is a square matrix. Then

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

so $F$ is a contraction if

$$\|B\|=q<1.$$

Consider the system  $Ax=b$. Decompose $A$ as

$$A=D\,(L+I+U)$$

where $D$ is a diagonal matrix, $L$ is strictly lower triangular matrix and $U$ is strictly upper triangular matrix.

### Jacobi method

Let

$$
B=-(L+U), \quad c=D^{-1}b.$$


If the matrix $A$ is __strictly diagonally dominant__ , 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,$$

then the map $F$ is a contraction (here it is possible to take other norma, as well) so the sequence

$$
x_{k+1}=-(L+U)x_k+c$$

converges towards the solution of the system $x$.

### Gauss-Seidel method 

Let

$$
B=-(I+L)^{-1}U, \quad c=(I+L)^{-1}\, D^{-1}b.
$$

Without proof we state the following: if the matrix $A$ is strictly diagonally dominant, then the map $F$ is a contraction, and the sequence

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

or

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

converges towards the solution $x$.

In [1]:
using LinearAlgebra
function myjacobi(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
import Random
Random.seed!(123)
n=8
A=rand(n,n)
# Let us make the matrix diagonally dominant
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.9249649192337484
 0.34893902341348704
 0.7844226234383731
 0.8497747226798917
 0.11992936825521072
 0.8601428096736736
 0.5768075478278709
 0.17805867488523286

In [3]:
# Starting vector
x₀=rand(n)

8-element Array{Float64,1}:
 0.9519169935552045
 0.8780997467517853
 0.009166252824636123
 0.5958963124776646
 0.9543508839651555
 0.8419452294952618
 0.2961019161242846
 0.8557272304345225

In [4]:
x,d=myjacobi(A,b,x₀)
# Solution
x

8-element Array{Float64,1}:
  0.08980305378762726
  0.014317578796243843
  0.07456867749469606
  0.0821775198466023
 -0.006931841179289444
  0.08111609269051698
  0.056856588309506684
 -0.005698840205529316

In [5]:
# Norm of the difference of two final iterations
d

1.6361911825413245e-13

In [6]:
# Residual
r=A*x-b

8-element Array{Float64,1}:
 -3.3995029014022293e-13
 -4.472533454702443e-13
 -3.397282455352979e-13
 -3.3006930522105904e-13
 -3.97237798210881e-13
 -5.331290964250002e-13
 -2.73003841755326e-13
 -4.266031972122164e-13

In [7]:
# Let us check the norm of the relative residual
norm(r)/(norm(A)*norm(x))

2.6299688420966703e-13

In [8]:
function mygaussseidel(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    # @show norm(U,Inf)
    y=Vector{Float64}(undef,n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [9]:
x,d=mygaussseidel(A,b,x₀)
# Solution
x

8-element Array{Float64,1}:
  0.08980305378765456
  0.01431757879628013
  0.07456867749472151
  0.08217751984662743
 -0.006931841179257599
  0.08111609269056345
  0.056856588309531365
 -0.005698840205490856

In [10]:
# Norm of the difference of two final iterations
d

7.735947486444373e-14

In [11]:
# Residual
A*x-b

8-element Array{Float64,1}:
 -5.773159728050814e-15
 -2.930988785010413e-14
 -2.6645352591003757e-14
 -1.7430501486614958e-14
 -6.7307270867900115e-15
 -3.552713678800501e-15
 -2.220446049250313e-16
  2.7755575615628914e-17

Let us measure the speed for larger matrices:

In [12]:
n=1024
A=rand(n,n)
A=A+n*I
b=rand(n)
x₀=rand(n)

1024-element Array{Float64,1}:
 0.16641614093330026
 0.602302264828624
 0.466840691287818
 0.39403858534532366
 0.0791761643861233
 0.13658772904085992
 0.6473994571097141
 0.18238250664440225
 0.29197913832578726
 0.10273804645853324
 0.3907351461765771
 0.8087040779127272
 0.6398599234494022
 ⋮
 0.05098896595273139
 0.26938136455069683
 0.9663749687160927
 0.6569570021468696
 0.6384179322464334
 0.6936973328260423
 0.5029705029470937
 0.16629793407793225
 0.7268488402721949
 0.18606528684826107
 0.7721857392292089
 0.8819063375259617

In [15]:
@time mygaussseidel(A,b,x₀);

  0.066748 seconds (69 allocations: 64.413 MiB, 7.85% gc time)


In [14]:
@time A\b;

  0.055581 seconds (15 allocations: 8.017 MiB, 21.32% gc time)


__Problem.__ Try to rewrite our functions to allocate less memory.